In [ ]:
# =====================================================
# FAKE NEWS DETECTION – DistilBERT + LoRA x LIAR Dataset
# =====================================================

# 1. CÀI ĐẶT
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"✅ Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CẢNH BÁO: Đang chạy trên CPU! Hãy chuyển sang T4 GPU.")

# 3. MOUNT DRIVE
drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/LIAR_DistilBERT_LoRA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. TẢI DATASET LIAR
print("\n⏳ Đang tải dataset LIAR...")
try:
    dataset = load_dataset("chengxuphd/liar2")
except Exception as e:
    print(f"⚠️ Lỗi tải bản 'liar2' ({e}), chuyển sang bản gốc 'liar'...")
    dataset = load_dataset("liar")

# Gộp train/val/test để xử lý nhất quán
df_train = pd.DataFrame(dataset['train'])
df_val = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])
df = pd.concat([df_train, df_val, df_test], ignore_index=True)

print(f"Tổng số mẫu ban đầu: {len(df)}")

# 5. XỬ LÝ DỮ LIỆU ĐẶC THÙ CHO LIAR (Quan trọng)

# A. Gom nhãn (6 -> 2)
def map_liar_labels(label):
    # Fake (0): false, barely-true, pants-fire
    if label in [0, 4, 5]: return 0
    # Real (1): half-true, mostly-true, true
    elif label in [1, 2, 3]: return 1
    return 0

df['label'] = df['label'].apply(map_liar_labels)

# B. Feature Engineering (Ghép ngữ cảnh)
# LIAR rất cần ngữ cảnh vì câu nói thường ngắn
def create_liar_content(row):
    stmt = str(row.get('statement', ''))
    subject = str(row.get('subject', ''))
    speaker = str(row.get('speaker', ''))
    party   = str(row.get('party_affiliation', ''))
    context = str(row.get('context', ''))

    # Cấu trúc: Statement [SEP] Subject | Speaker (Party) | Context
    return f"{stmt} [SEP] {subject} | {speaker} ({party}) | {context}"

print("🛠️ Đang tạo nội dung đầu vào (Statement + Context)...")
df['content'] = df.apply(create_liar_content, axis=1)

# C. Làm sạch
def clean_text(text):
    if not isinstance(text, str): return ""
    t = text.lower()
    t = re.sub(r'https?://\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    # Giữ lại dấu câu quan trọng cho ngữ cảnh
    t = re.sub(r'[^a-zA-Z0-9\s\.\,\-\(\)]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 10] # Loại bỏ câu quá ngắn

# Phân bố & Class Weights
label_counts = df['label'].value_counts(normalize=True)
print(f"Phân bố sau xử lý: {label_counts.to_dict()}")

classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 6. SPLIT DATA
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 7. TOKENIZER
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("⚙️ Tokenizing...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. LOAD MODEL & APPLY LoRA
print(f"\n🚀 Tải model {MODEL_NAME} và áp dụng LoRA...")
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
base_model.config.id2label = {0: "Fake", 1: "Real"}
base_model.config.label2id = {"Fake": 0, "Real": 1}

# Cấu hình LoRA cho DistilBERT
# DistilBERT sử dụng tên module: q_lin, k_lin, v_lin
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,                   
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_lin", "k_lin", "v_lin"]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# 9. CUSTOM TRAINER (Weighted Loss)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        w = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 10. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,                 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,                
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 11. TRAIN
print("\n🔥 BẮT ĐẦU HUẤN LUYỆN (DistilBERT + LoRA trên LIAR)...")
trainer.train()

print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
print(trainer.evaluate(tokenized["test"]))

# Save Adapter
final_path = os.path.join(OUTPUT_DIR, "final_liar_lora")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"\n✅ Đã lưu LoRA Adapter tại: {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.9 MB/s eta 0:00:00
✅ Device: Tesla T4 | VRAM: 15.8 GB
Mounted at /content/drive

⏳ Đang tải dataset LIAR...


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,034,498 || all params: 67,989,508 || trainable%: 1.5216

🔥 BẮT ĐẦU HUẤN LUYỆN (DistilBERT + LoRA trên LIAR)...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.689900,0.688236,0.543554,0.544365,0.592194,0.543554
2,0.690100,0.686918,0.598868,0.579766,0.579613,0.598868
3,0.687100,0.685273,0.604094,0.581406,0.583573,0.604094
4,0.680200,0.687187,0.586672,0.582375,0.579805,0.586672
5,0.675700,0.682815,0.579704,0.582722,0.587600,0.579704



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.6805388331413269, 'eval_accuracy': 0.5824989116238573, 'eval_f1': 0.5850792225295239, 'eval_precision': 0.5889396143904052, 'eval_recall': 0.5824989116238573, 'eval_runtime': 1.4719, 'eval_samples_per_second': 1560.612, 'eval_steps_per_second': 48.918, 'epoch': 5.0}

✅ Đã lưu LoRA Adapter tại: /content/drive/MyDrive/LIAR_DistilBERT_LoRA/final_liar_lora
